In [3]:
pip show langchain_openai

Name: langchain-openai
Version: 0.1.23
Summary: An integration package connecting OpenAI and LangChain
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\miniconda\Gen_AI_Automation_Deep_Learning\LangChain_Mastery_Develop_LLM_Apps_with_LangChain_&_Pinecone\venv\Lib\site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get("OPENAI_API_KEY")

True

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
output = llm.invoke("Explain Quantum Mechanics in one sentence", model = "gpt-4o-mini")
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the smallest scales, where particles exhibit both wave-like and particle-like properties, and their states are fundamentally probabilistic rather than deterministic.


In [5]:
from langchain.schema import(
    SystemMessage, 
    AIMessage, 
    HumanMessage
)
messages = [
    SystemMessage(content = "You are a physicist and respond only in German"),
    HumanMessage(content="Explain quantum mechanics in one sentence")
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch probabilistische Wellenfunktionen.


## Caching LLM Responses
- Using Langchain's `InMemoryCache()`
- Using `SQLiteCache()`

In [6]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")

In [7]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = "Explain quantum mechanics in 5 words"
output = llm.invoke(prompt)

CPU times: total: 31.2 ms
Wall time: 1.32 s


In [8]:
print(output)



Wave-like behavior of subatomic particles.


In [9]:
%%time
llm.invoke(prompt)

CPU times: total: 0 ns
Wall time: 0 ns


'\n\nWave-like behavior of subatomic particles.'

In [10]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path = ".langchain.db"))

In [11]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: total: 0 ns
Wall time: 717 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired. "

In [12]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: total: 344 ms
Wall time: 361 ms


" Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired. "